In [1]:
import pandas as pd
from keras.layers import LSTM, Dense, Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.models import Sequential
from keras import metrics

In [ ]:
train_df = pd.read_csv("https://raw.githubusercontent.com/elip06/covid19-fact-checking/main/dataset_preparation/preprocessed_datasets/train_dataset_final.csv?token=AL3S7USONU7SWCL464X7SOLA34FCO")
test_df = pd.read_csv("https://raw.githubusercontent.com/elip06/covid19-fact-checking/main/dataset_preparation/preprocessed_datasets/test_dataset_final.csv?token=AL3S7USJS5CMY5SRWUR4ED3A34FEW")
X_train = train_df.text
Y_train = train_df.labels
X_test = test_df.text
Y_test = test_df.labels

In [ ]:
max_words = 5000
max_len = 512
tokenizer_lstm = Tokenizer(num_words=max_words)
tokenizer_lstm.fit_on_texts(X_train)
sequences = tokenizer_lstm.texts_to_sequences(X_train)
X_train_lstm = sequence.pad_sequences(sequences,maxlen=max_len)

In [ ]:
embedding_vector_length = 32
lstm_model = Sequential()
lstm_model.add(Embedding(max_words, embedding_vector_length, input_length=max_len))
lstm_model.add(LSTM(100))
lstm_model.add(Dense(1, activation='sigmoid'))
lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', metrics.FalsePositives(), metrics.FalseNegatives(), metrics.TruePositives(), metrics.TrueNegatives()])
print(lstm_model.summary())
lstm_model.fit(X_train_lstm, Y_train, epochs=4, batch_size=64)

In [ ]:
test_sequences = tokenizer_lstm.texts_to_sequences(X_test)
X_test_lstm = sequence.pad_sequences(test_sequences,maxlen=max_len)
accr = lstm_model.evaluate(X_test_lstm, Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}\n  False Positives: {:0.0f}\n  False Negatives: {:0.0f}\n  True Positives: {:0.0f}\n  True Negatives: {:0.0f}'.format(accr[0],accr[1], accr[2], accr[3], accr[4], accr[5]))